# Simple Sequence to Sequence model 



https://github.com/golbin/TensorFlow-Tutorials/blob/master/08%20-%20RNN/03%20-%20Seq2Seq.py

https://www.tensorflow.org/api_docs/python/tf/contrib/nn

In [1]:
import tensorflow as tf 
import numpy as np

In [2]:
#사전 구성
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [3]:
#영어 -> 한글 번역 학습 데이터 
seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

In [4]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data: 
        input = [num_dic[n] for n in seq[0]]#단어를 구성하는 철자의 id를 리스트에 저장
        output = [num_dic[n] for n in ('S'+seq[1])]#target 단어를 구성하는 철자의 id를 저장
        target = [num_dic[n] for n in (seq[1]+'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
        
    return input_batch, output_batch, target_batch


In [5]:
# 옵션설정 
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
n_class = n_input = dic_len

In [14]:
#신경망 모델 구성 

#seq2seq model에서 인코더의 입력과 디코더의 입력의 형식은 같다. 
#[batcxh size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])

targets = tf.placeholder(tf.int64, [None, None])

#encoder cell 구성 
with tf.variable_scope('encode'):
    enc_cell = tf.contrib.rnn.BasicRNNCell(n_hidden)#128개 rnn cell 생성
    enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob = 0.5)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype = tf.float32)
    
#decoder cell 구성 
with tf.variable_scope('decode'): 
    dec_cell = tf.contrib.rnn.BasicRNNCell(n_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob = 0.5)
    
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell,dec_input, initial_state=enc_states, 
                                            dtype = tf.float32)
    
    
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits = model, labels = targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

ValueError: Variable encode/rnn/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-7-9dee8ad11f9b>", line 15, in <module>
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype = tf.float32
  File "/home/junhee/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/junhee/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


**함수 설명**

#### tf.variable_scope()
: tensorflow에서 제공하는 변수를 공유해서 쓸수 있는 기능, 같은 변수 파라미터를 넘겨주고 싶을 때 사용된다. 
![''](variable_scope.png)

#### tf.contrib.rnn.BasicRNNCell()
: 가장 기본적인 RNN Cell을 말함. 
https://www.tensorflow.org/api_docs/python/tf/contrib/rnn (이외 cell 참조)

#### tf.contrib.rnn.DropoutWrapper()
:주어진 cell에서 입출력에 누락을 추가 시킴.

#### tf.nn.dynamic_rnn()


In [8]:
# 신경망 모델 학습 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _,loss = sess.run([optimizer, cost],
                     feed_dict = {enc_input : input_batch,
                                  dec_input : output_batch,
                                  targets : target_batch})
    print('Epoch: ',epoch+1)
    print('Cost: ',format(loss))
    
print('완료')

Epoch:  1
Cost:  3.6922895908355713
Epoch:  2
Cost:  2.546456813812256
Epoch:  3
Cost:  1.8289705514907837
Epoch:  4
Cost:  1.102649450302124
Epoch:  5
Cost:  0.8733641505241394
Epoch:  6
Cost:  0.6365712881088257
Epoch:  7
Cost:  0.4792424440383911
Epoch:  8
Cost:  0.2619868516921997
Epoch:  9
Cost:  0.14913156628608704
Epoch:  10
Cost:  0.06083295866847038
Epoch:  11
Cost:  0.1275276094675064
Epoch:  12
Cost:  0.10499496012926102
Epoch:  13
Cost:  0.11083739250898361
Epoch:  14
Cost:  0.13728494942188263
Epoch:  15
Cost:  0.14143258333206177
Epoch:  16
Cost:  0.038996487855911255
Epoch:  17
Cost:  0.031926944851875305
Epoch:  18
Cost:  0.07124059647321701
Epoch:  19
Cost:  0.033267974853515625
Epoch:  20
Cost:  0.026116998866200447
Epoch:  21
Cost:  0.022883199155330658
Epoch:  22
Cost:  0.020257113501429558
Epoch:  23
Cost:  0.026245979592204094
Epoch:  24
Cost:  0.004696179181337357
Epoch:  25
Cost:  0.005607190076261759
Epoch:  26
Cost:  0.012052875012159348
Epoch:  27
Cost:  0.00

In [9]:
#번역 테스트 

def translate(word):
    seq_data = [word, 'P'*len(word)]
    
    input_batch, output_batch, target_batch = make_batch([seq_data])
    
    prediction = tf.argmax(model, 2)
    
    result = sess.run(prediction, 
                     feed_dict={enc_input: input_batch,
                               dec_input : output_batch,
                               targets : target_batch})
    
    decoded = [char_arr[i] for i in result[0]]
    
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    
    return translated

In [10]:
#test start 
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))

word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑


ValueError: 'E' is not in list